<style>
    div.output_scroll {
    height: 1200px;
}
</style>

In [1]:
# Import all required libraries
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import pandas as pd
import matplotlib.pyplot as plt
import urllib.parse

# Import your CRUD module
from AnimalShelter import AnimalShelter  # Provides access to MongoDB with CRUD operations


C:\Users\jblat\AppData\Roaming\Python\Python313\site-packages\dash\dash.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, parse_version


In [2]:
# Instantiate the AnimalShelter object using local MongoDB setup (no username/password needed now)
shelter = AnimalShelter()

# Read all records from the MongoDB collection and convert them into a DataFrame
df = pd.DataFrame.from_records(shelter.read({}))

# Drop the MongoDB '_id' field if it exists, as it's not needed for display
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Preview the first few records to verify the data
df.head()


Unnamed: 0 age_upon_outcome animal_id animal_type                    breed  \
0           3          2 years   A716330         Dog  Chihuahua Shorthair Mix   
1          16          5 years   A723742         Dog  Miniature Schnauzer Mix   
2          14          2 years   A742287         Dog        Boxer/Bullmastiff   
3          12           1 year   A664843         Dog             Pit Bull Mix   
4          17         6 months   A668960         Dog             Pit Bull Mix   

                 color date_of_birth             datetime  \
0          Brown/White    2013-11-18  2015-12-28 18:43:00   
1          Black/White    2011-04-05  2016-04-10 17:27:00   
2  Brown Brindle/White    2015-01-18  2017-02-11 12:30:00   
3          Brown/White    2013-06-09  2014-08-18 17:24:00   
4           Blue/White    2013-06-12  2013-12-27 16:56:00   

             monthyear      name outcome_subtype outcome_type  \
0  2015-12-28T18:43:00     Frank            None     Adoption   
1  2016-04-10T17:27:00  Gretchen            None     Adoption   
2  2017-02-11T12:30:00    *Kawhi            None     Adoption   
3  2014-08-18T17:24:00  Sherlock         Partner     Transfer   
4  2013-12-27T16:56:00     *Gigi            None     Adoption   

  sex_upon_outcome  location_lat  location_long  age_upon_outcome_in_weeks  
0    Neutered Male     30.759575     -97.552375                 110.111409  
1    Spayed Female     30.479288     -97.408853                 261.818155  
2    Neutered Male     30.455115     -97.308778                 107.931548  
3    Neutered Male     30.451555     -97.474105                  62.246429  
4    Spayed Female     30.594341     -97.248993                  28.386508

In [3]:
df.head()


Unnamed: 0 age_upon_outcome animal_id animal_type                    breed  \
0           3          2 years   A716330         Dog  Chihuahua Shorthair Mix   
1          16          5 years   A723742         Dog  Miniature Schnauzer Mix   
2          14          2 years   A742287         Dog        Boxer/Bullmastiff   
3          12           1 year   A664843         Dog             Pit Bull Mix   
4          17         6 months   A668960         Dog             Pit Bull Mix   

                 color date_of_birth             datetime  \
0          Brown/White    2013-11-18  2015-12-28 18:43:00   
1          Black/White    2011-04-05  2016-04-10 17:27:00   
2  Brown Brindle/White    2015-01-18  2017-02-11 12:30:00   
3          Brown/White    2013-06-09  2014-08-18 17:24:00   
4           Blue/White    2013-06-12  2013-12-27 16:56:00   

             monthyear      name outcome_subtype outcome_type  \
0  2015-12-28T18:43:00     Frank            None     Adoption   
1  2016-04-10T17:27:00  Gretchen            None     Adoption   
2  2017-02-11T12:30:00    *Kawhi            None     Adoption   
3  2014-08-18T17:24:00  Sherlock         Partner     Transfer   
4  2013-12-27T16:56:00     *Gigi            None     Adoption   

  sex_upon_outcome  location_lat  location_long  age_upon_outcome_in_weeks  
0    Neutered Male     30.759575     -97.552375                 110.111409  
1    Spayed Female     30.479288     -97.408853                 261.818155  
2    Neutered Male     30.455115     -97.308778                 107.931548  
3    Neutered Male     30.451555     -97.474105                  62.246429  
4    Spayed Female     30.594341     -97.248993                  28.386508

In [4]:
# Create and configure the Dash app
app = JupyterDash('ProjectTwoDashboard')

# Define the layout of the dashboard
app.layout = html.Div([
    # Header Image
    html.Img(src='assets/Glogo.png', style={'height': '100px'}),

    # Dashboard Title
    html.Center(html.H1("Project Two Dashboard - Steven Blathras")),

    # Filter section: Rescue Type
    html.H4("Rescue Type Filter"),
    dcc.RadioItems(
        id='rescue-type',
        options=[
            {'label': 'Water Rescue', 'value': 'Water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster'},
            {'label': 'Reset', 'value': 'All'}
        ],
        value='All',
        labelStyle={'display': 'block'},
        inputStyle={"margin-right": "10px"}
    ),

    html.Hr(),

    # Data table of filtered animals
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        row_selectable='single',
        selected_rows=[0],
        style_table={'overflowX': 'auto'},
        style_cell={
            'textAlign': 'left',
            'padding': '5px',
            'whiteSpace': 'normal'
        }
    ),

    html.Br(),

    # Map location of the selected animal
    html.Div(id='map-id', className='col s12 m6'),

    html.Br(),

    # Breed chart visualization
    dcc.Graph(id='breed-chart'),

    html.Center(html.B("Steven Blathras - SNHU CS-340 MongoDB Dashboard"))
])


C:\Users\jblat\AppData\Roaming\Python\Python313\site-packages\dash\dash.py:516: UserWarning: JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.
  warnings.warn(


In [5]:
# Callback to update table, map, and chart based on user inputs
@app.callback(
    [Output('datatable-id', 'data'),
     Output('map-id', 'children'),
     Output('breed-chart', 'figure')],
    [Input('rescue-type', 'value'),
     Input('datatable-id', 'selected_rows')]
)
def update_dashboard(rescue_type, selected_rows):
    # Filter for dogs under 2 years old (104 weeks)
    dog_age_limit = 104
    query = {
        "animal_type": "Dog",
        "age_upon_outcome_in_weeks": {"$lte": dog_age_limit}
    }

    # Define breeds by rescue type
    rescue_breeds = {
        "Water": ["Labrador Retriever", "Chesapeake Bay Retriever", "Newfoundland"],
        "Mountain": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog"],
        "Disaster": ["Doberman Pinscher", "German Shepherd", "Golden Retriever"],
        "All": []
    }

    # Apply breed filter if a specific type is selected
    if rescue_type != "All":
        query["breed"] = {"$in": rescue_breeds[rescue_type]}

    # Retrieve and clean data
    results = shelter.read(query)
    dff = pd.DataFrame.from_records(results)
    if '_id' in dff.columns:
        dff.drop(columns=['_id'], inplace=True)

    # Handle empty results
    if dff.empty:
        return [], [html.P("No data available.")], {
            'data': [],
            'layout': {'title': 'No data available'}
        }

    # Validate selected row index
    row = selected_rows[0] if selected_rows else 0
    row = min(row, len(dff) - 1)  # Ensure row is within bounds

    # Create map component
    map_output = [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[30.75, -97.48], zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[
                        dff.iloc[row].get("location_lat", 30.75),
                        dff.iloc[row].get("location_long", -97.48)
                    ],
                    children=[
                        dl.Tooltip(dff.iloc[row].get("breed", "Unknown")),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(dff.iloc[row].get("name", "Unknown"))
                        ])
                    ]
                )
            ]
        )
    ]

    # Generate bar chart of breed counts
    breed_counts = dff['breed'].value_counts()
    chart_fig = {
        'data': [{
            'x': breed_counts.index,
            'y': breed_counts.values,
            'type': 'bar',
            'marker': {'color': 'teal'}
        }],
        'layout': {
            'title': 'Dog Breeds by Count (Filtered)',
            'xaxis': {'title': 'Breed'},
            'yaxis': {'title': 'Count'}
        }
    }

    return dff.to_dict('records'), map_output, chart_fig


In [6]:
# Import required libraries
from dash import Dash, dcc, html, dash_table
from dash.dependencies import Input, Output
import dash_leaflet as dl
import pandas as pd
import urllib.parse

# Import your CRUD module
from AnimalShelter import AnimalShelter

# Set up MongoDB credentials and connect
username = urllib.parse.quote_plus("aacuser")
password = urllib.parse.quote_plus("Blathras")
shelter = AnimalShelter(username, password)
df = pd.DataFrame.from_records(shelter.read({}))

# Clean up the data
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Initialize Dash app
app = Dash(__name__)

# Get unique breeds for dropdown
breed_options = [{'label': breed, 'value': breed} for breed in sorted(df['breed'].dropna().unique())]

# Define layout
app.layout = html.Div([
    html.H1("Animal Shelter Dashboard", style={'textAlign': 'center'}),
    html.P("Use the dropdown below to filter animals by breed:", style={'textAlign': 'center'}),

    dcc.Dropdown(
        id='breed-dropdown',
        options=[{'label': 'All Breeds', 'value': 'All'}] + breed_options,
        value='All',
        style={'width': '50%', 'margin': '0 auto'}
    ),

    html.Br(),

    dash_table.DataTable(
        id='datatable',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
        style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'}
    )
])

# Callback to filter data
@app.callback(
    Output('datatable', 'data'),
    Input('breed-dropdown', 'value')
)
def update_table(selected_breed):
    if selected_breed == 'All':
        filtered_df = df
    else:
        filtered_df = df[df['breed'] == selected_breed]
    return filtered_df.to_dict('records')

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
